# Task -> Create a custom chatbot using Langchain:

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader as loader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from flask import Flask,request,jsonify
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

#### Extract data from https://brainlox.com/courses/category/technical using URL loaders from Langchain

In [2]:
url="https://brainlox.com/courses/category/technical"
data=loader(urls=[url]).load()

#### Cerate embeddings and store it in a vector-store

In [ ]:
embed=HuggingFaceEmbeddings()
vectors=FAISS.from_documents(data,embed) 

#### Create a FLASK RESTFUL API to handle the conversation

In [ ]:
app=Flask(__name__)
queryChain=RetrievalQA.from_chain_type(llm=ChatOpenAI(),chain_type="stuff",retriever=vectors.as_retriever())
@app.route('/chat', methods=['POST'])
def chat():
    try:
        question=request.json['question']
        answer=queryChain.run(question)
        return jsonify({'answer ':answer})
    except Exception:
        return jsonify({"error 500": str(Exception)})

if __name__ =='__main__':
    app.run(debug=True)